In [ ]:
!pip install underground

In [ ]:
import os
os.environ['MTA_API_KEY'] = "HoYzecGBd54ZYC8bHLOgq2aBX6lZTu1n5NIX1ETm"


In [ ]:
import os

from underground import metadata, SubwayFeed

API_KEY = os.getenv('MTA_API_KEY')
ROUTE = 'L'
feed = SubwayFeed.get(ROUTE, api_key=API_KEY)

# request will read from $MTA_API_KEY if a key is not provided
feed = SubwayFeed.get(ROUTE)

# under the hood, the Q route is mapped to a URL. This call is equivalent:
#URL = 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw'
#feed = SubwayFeed.get(URL)

# or
URL = metadata.resolve_url(ROUTE)
feed = SubwayFeed.get(URL)

In [ ]:
stop_dict = feed.extract_stop_dict()

In [ ]:
stop_dict["L"]["L16N"]

[datetime.datetime(2022, 5, 7, 16, 22, 36, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
 datetime.datetime(2022, 5, 7, 16, 30, 2, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
 datetime.datetime(2022, 5, 7, 16, 38, 2, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
 datetime.datetime(2022, 5, 7, 16, 46, 2, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
 datetime.datetime(2022, 5, 7, 16, 54, 2, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)]

In [ ]:
from datetime import datetime
now = datetime.now() # current date and time

time = now.strftime("%H:%M:%S")
print("time:", time)

time: 20:27:11


In [ ]:
feed = SubwayFeed.get('L')

In [ ]:
print(feed)

header=FeedHeader(gtfs_realtime_version='1.0', timestamp=datetime.datetime(2022, 5, 7, 19, 57, 6, tzinfo=datetime.timezone.utc)) entity=[Entity(id='1', vehicle=None, trip_update=TripUpdate(trip=Trip(trip_id='093250_L..S', start_time=None, start_date=20220507, route_id='L'), stop_time_update=[StopTimeUpdate(stop_id='L20S', arrival=UnixTimestamp(time=datetime.datetime(2022, 5, 7, 19, 56, 45, tzinfo=datetime.timezone.utc)), departure=UnixTimestamp(time=datetime.datetime(2022, 5, 7, 19, 57, tzinfo=datetime.timezone.utc))), StopTimeUpdate(stop_id='L21S', arrival=UnixTimestamp(time=datetime.datetime(2022, 5, 7, 19, 58, 2, tzinfo=datetime.timezone.utc)), departure=UnixTimestamp(time=datetime.datetime(2022, 5, 7, 19, 58, 17, tzinfo=datetime.timezone.utc))), StopTimeUpdate(stop_id='L22S', arrival=UnixTimestamp(time=datetime.datetime(2022, 5, 7, 20, 1, 8, tzinfo=datetime.timezone.utc)), departure=None)])), Entity(id='2', vehicle=Vehicle(trip=Trip(trip_id='093250_L..S', start_time=None, start_dat

In [ ]:
! underground stops Q | grep Q05S

Q05S 16:01 16:09 16:17 16:25 16:33 16:41 16:49 16:57


In [ ]:
! underground findstops parkside

ID: D27N    Direction: NORTH    Lat/Lon: 40.655292,-73.961495    Name: PARKSIDE AV
ID: D27S    Direction: SOUTH    Lat/Lon: 40.655292,-73.961495    Name: PARKSIDE AV


In [ ]:
from google.transit import gtfs_realtime_pb2
import requests
import time # imports module for Epoch/GMT time conversion
import os # imports package for dotenv
from dotenv import load_dotenv, find_dotenv # imports module for dotenv
load_dotenv(find_dotenv()) # loads .env from root directory

# The root directory requires a .env file with API_KEY assigned/defined within
# and dotenv installed from pypi. Get API key from http://datamine.mta.info/user
api_key = os.environ['API_KEY']

# Requests subway status data feed from City of New York MTA API
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('http://datamine.mta.info/mta_esi.php?key={}&feed_id=21'.format(api_key))
feed.ParseFromString(response.content)

# The MTA data feed uses the General Transit Feed Specification (GTFS) which
# is based upon Google's "protocol buffer" data format. While possible to
# manipulate this data natively in python, it is far easier to use the
# "pip install --upgrade gtfs-realtime-bindings" library which can be found on pypi
from protobuf_to_dict import protobuf_to_dict
subway_feed = protobuf_to_dict(feed) # subway_feed is a dictionary
realtime_data = subway_feed['entity'] # train_data is a list

# Because the data feed includes multiple arrival times for a given station
# a global list needs to be created to collect the various times
collected_times = []

# This function takes a converted MTA data feed and a specific station ID and
# loops through various nested dictionaries and lists to (1) filter out active
# trains, (2) search for the given station ID, and (3) append the arrival time
# of any instance of the station ID to the collected_times list
def station_time_lookup(train_data, station):
    for trains in train_data: # trains are dictionaries
        if trains.get('trip_update', False) != False:
            unique_train_schedule = trains['trip_update'] # train_schedule is a dictionary with trip and stop_time_update
            unique_arrival_times = unique_train_schedule['stop_time_update'] # arrival_times is a list of arrivals
            for scheduled_arrivals in unique_arrival_times: #arrivals are dictionaries with time data and stop_ids
                if scheduled_arrivals.get('stop_id', False) == station:
                    time_data = scheduled_arrivals['arrival']
                    unique_time = time_data['time']
                    if unique_time != None:
                        collected_times.append(unique_time)

ModuleNotFoundError: ignored